# Compressing the System

## Overview

### Questions

* How do I compress the system to a target density? 
* What is a packing fraction?

### Objectives

* Show how to compute the **packing fraction** of a system.
* Explain how how an **Updater** is an **operation** that modifies the system when its **Trigger** returns `True`.
* Demonstrate using the **QuickCompress** updater to achieve a target packing fraction.
* Demonstrate using the **MoveSize** tuner to adjust the trial move size.
* Show how operations can **complete** the simulation run.

In [1]:
import hoomd
import math
import copy

## Packing fraction

Self-assembly in the hard particle systems typically occurs at a **packing fraction** above 0.5.
The **packing fraction** is the ratio of the volume occupied by the particles to the volume of the **periodic box**.
So far in this tutorial, we have **randomized** a system of *N* octahedra in a box with a very low packing fraction and stored that in `random.gsd`.
Let's initialize a **Simulation** with this configuration and see what packing fraction it is at.

In [2]:
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu)
sim.create_state_from_gsd(filename='random.gsd')

HOOMD-blue v2.9.0-1828-gd903f0dc6 DOUBLE HPMC_MIXED SSE SSE2 SSE3 SSE4_1 SSE4_2 AVX AVX2 
Compiled: 06/10/2020
Copyright (c) 2009-2019 The Regents of the University of Michigan.
HOOMD-blue is running on the CPU


Compute the [volume of our octahedron](https://en.wikipedia.org/wiki/Octahedron):

In [3]:
a = math.sqrt(2)/2
V_particle = 1/3 * math.sqrt(2) * a**3

Compute the **packing fraction** (TODO: Add number of particles to state?)

In [4]:
initial_packing_fraction = sim.state.snapshot.particles.N * V_particle / sim.state.box.volume
print(initial_packing_fraction)

0.09126759678492466


As you can see, this **packing fraction** is very low and the box volume needs to be significantly reduced to achieve a **packing fraction** above 0.5.

We will use HPMC to move particles into non-overlapping configurations while we compress the system.
Set up the HPMC integrator for the octahedron simulations:

In [5]:
mc = hoomd.hpmc.integrate.ConvexPolyhedron(seed=42)
mc.shape['octahedron'] = dict(vertices=[(-0.5, 0, 0),
                                         (0.5, 0, 0),
                                         (0, -0.5, 0),
                                         (0, 0.5, 0),
                                         (0, 0, -0.5),
                                         (0, 0, 0.5)])
sim.operations.integrator = mc

## The QuickCompress updater

An **Updater** is a type of **operation** in HOOMD-blue that makes changes to the **state**.
To use an **Updater**, we need to instantiate the object, assign a **Trigger**, and add it to the **Simulation**.
**Simulation** will apply the **Updater** on **time steps** where the **Trigger** is `True`.
The most commonly used **Trigger** is **PeriodicTrigger** which executes the **Updater** ever `period` steps.

**QuickCompress** is an **Updater** that works with HPMC to quickly compress the box to a target volume.
During compression, **QuickCompress** reduces the box volume by a scale factor, while *allowing slight overlaps between the particles*.
It then waits for the translation and rotation **trial moves** to remove these overlaps before it reduces the volume again.
This process temporarily produces invalid system configurations, but is much quicker than a process that does not allow temporary overlaps.

Compute the final box size with a **packing fraction** above 0.5 and configure a **QuickCompress** to **trigger** every 10 **time steps**.

In [6]:
initial_box = sim.state.box
final_box = hoomd.Box.from_box(initial_box)
final_packing_fraction = 0.55
final_box.volume = sim.state.snapshot.particles.N * V_particle / final_packing_fraction
# TODO: Implement QuickCompress
# compress = hoomd.hpmc.update.QuickCompress(trigger=hoomd.trigger.Periodic(10), target_box = final_box)

Add the **Updater** to the **Simulation**:

In [7]:
# sim.operations.updaters.append(compress)

## The TrialMoveSize tuner

An **Tuner** is another type of **operation**.
**Tuners** make changes to other **operations** or the system **state** to improve performance.
In HPMC, the translation and rotation trial move sizes have a huge impact on the performance of the system.
Make the move sizes too small and it takes many time steps to make appreciable changes to the system.
Make the move sizes too large leads to a small **acceptance ratio** and it again takes many time steps to make appreciable changes.
In most cases, the optimal **acceptance ratio** is 20%.
The **TrialMoveSize** tuner monitors the **acceptance ratio** and adjusts `d` and `a` to achieve this target.

The optimal move size depends on the density of the system.
Use the **TrialMoveSize** tuner to adjust the move sizes regularly during compression:

In [8]:
# TODO: Implement the tuner in the v3 API
# tune = hoomd.hpmc.tune.TrialMoveSize(integrator=mc, tune=['a', 'd'], trigger=hoomd.trigger.Periodic(10))
# sim.operations.tuners.append(tune)

## Run until complete

The **QuickCompress** updater will **complete** the simulation run when it reaches the target box size.
This will stop the run before the requested number of **time steps** is executed.
Run for enough steps to complete the compression:

In [9]:
# TODO: implement this
# sim.run(1e6)

Check to see if the compression completed successfully:

In [10]:
# TODO: implement this
# if not compress.completed:
#     raise RuntimeError("Compression failed to complete")

In [11]:
# This code is not part of the tutorial. 
# It is a temporary Python implementation of QuickCompress method. We will later re-implement in C++ as an Updater and use that here.
D = 2
max_scale = 0.99
max_overlaps = 82*4 / 4
mc.d['octahedron'] = 0.02693819753092484
mc.a['octahedron'] = 0.019871213815167245

import time
import numpy
start_time = time.time()

current_box = hoomd.Box.from_box(initial_box)

N = sim.state.snapshot.particles.N
sim.operations.schedule()

import sys

count = 0

while final_box.volume < current_box.volume:
    
    v = numpy.random.random(1)
    current_scale = v * max_scale + (1-v)*1

    prev_volume = current_box.volume
    current_box.volume = current_scale * current_box.volume

    if current_box.volume < final_box.volume:
        current_box.volume = final_box.volume

    hoomd.update.BoxResize.scale_state(sim.state, current_box)

    overlaps = mc.overlaps

    if overlaps > max_overlaps:
        current_box.volume = prev_volume
        hoomd.update.BoxResize.scale_state(sim.state, current_box)
        overlaps = mc.overlaps
        
        sim.run(10)
       
#     if sim.timestep % 100 == 0:
#         print("phi =", (N*V_particle) / (current_box.volume), ": overlaps =", overlaps);
    
    # run until all overlaps are removed
    while overlaps > 0:
        sim.run(10);
        
        overlaps = mc.overlaps;
        sys.stdout.flush();
        
        gamma = 2.0
        acceptance = mc.translate_moves[0] / sum(mc.translate_moves)
        d_scale = ((1.0 + gamma) * acceptance) / (0.2 + gamma * acceptance)
        mc.d['octahedron'] = mc.d['octahedron'] * d_scale
        max_scale = (3*1 + (1 - mc.d['octahedron']/D)**3)/4

        acceptance = mc.rotate_moves[0] / sum(mc.rotate_moves)
        a_scale = ((1.0 + gamma) * acceptance) / (0.2 + gamma * acceptance)
        mc.a['octahedron'] = mc.a['octahedron'] * a_scale

#print("phi =", (N*V_particle) / (current_box.volume), ": overlaps =", overlaps);
#end_time = time.time()
#print(end_time - start_time)

*Warning*: Box size is too small or move size is too large for the minimum image convention.
Testing 33 images per trial move, performance may slow.
This message will not be repeated.


Let's save the final configuration to a GSD file for use in the next stage of the simulation.

In [12]:
hoomd.dump.GSD.write_state(state=sim.state, filename='compressed.gsd')